# Stage directions: basic features

This notebook contains:

* directions extraction
* basic values calculation

## Imports
<span style="font-size:larger;color:blue;">Run this before everything else!</span>

In [1]:
# imports
import os
import re
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup
from statistics import mean
from pymystem3 import Mystem


# we need this namespaces parameter to search in TEI-encoded files correctly
tei_ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

# one Mystem for everything, so that we won't have to initialize it every time
mystem = Mystem()

## Preparation

### Data
I used [RusDraCor](https://github.com/dracor-org/rusdracor) as of 7th April, 2018. More precise data on the corpus is below.

In [2]:
corpus_path = ".." + os.sep + "data"

### Play class
I want to create a table with information on the plays we research, so it will be much easier to have a separate ```Play``` class with all the data.


```python
class Play:
    def __init__(self, path, author, title, year_written, year_published, amount_of_directions, amount_of_acts):
        self.path = path
        self.author = author
        self.title = title
        self.year_written = year_written
        self.year_published = year_published
        self.amount_of_directions = amount_of_directions
        self.amount_of_acts = amount_of_acts
```

Update: we don't really need that one.

### Folders

In [ ]:
if not os.path.exists(".." + os.sep)

## Extraction

### Play information
The majority of information pieces can be extracted from the ```html <teiHeader></teiHeader>``` in XML itself. Each act is wrapped in ```<div type="act">```, so counting the acts also isn't much of a problem.

In [3]:
def get_play_data(play_path):
    reg_space = re.compile("\s{2,}")
    root = etree.parse(play_path)
    title = root.find(".//tei:titleStmt/tei:title", tei_ns).text
    title = reg_space.sub(" ", title)
    author = root.find(".//tei:titleStmt/tei:author", tei_ns).text
    try:
        written_str = root.find(".//tei:sourceDesc/tei:bibl/tei:bibl/tei:date[@type=\"written\"]", tei_ns).attrib["when"]
        year_written = int(written_str)
    except:
        year_written = -1
    try:
        published_str = root.find(".//tei:sourceDesc/tei:bibl/tei:bibl/tei:date[@type=\"print\"]", tei_ns).attrib["when"]
        year_published = int(published_str)
    except:
        year_published = -1
    acts = len(root.findall(".//tei:text/tei:body/tei:div[@type=\"act\"]", tei_ns))
    return author, title, year_written, year_published, acts    

### Amount of words in a play
Word count is required to count some averages later on.

In [4]:
def word_count(play_path, mystem):
    with open(play_path, "r", encoding="utf-8") as play:
        soup = BeautifulSoup(play, "xml")
        text = soup.find("text").text
    lemmas_play = set(mystem.lemmatize(text))
    words_play = [item for item in mystem.analyze(text) if "analysis" in item]
    return len(lemmas_play), len(words_play)

### Directions
Basically, all the directions are wrapped by the ```<stage></stage>``` tag, so we can just use regex. The only problem with this approach is that sometimes directions are on separate lines, and many spaces and tabs may occur in between two words — and this can also be fixed with the help of regex. 

In [5]:
def get_directions(play_path):
    reg_space = re.compile("\s{2,}")
    root = etree.parse(play_path)
    directions_tags = root.findall(".//tei:stage", tei_ns)
    directions_text = []
    for direction in directions_tags:
        try:
            text = direction.text
            text = reg_space.sub(" ", text)
            text = text.replace("\xa0", " ")
            text = text.strip("\(\) \.")
            text = text.lower()
            directions_text.append(text)
        except:
            pass
    return directions_text

In addition — let's count the average length of a direction in a play!

In [6]:
def average_direction(directions, mystem):
    dir_length = []
    dir_lemmas = []
    for direction in directions:
        length = len([item for item in mystem.analyze(direction) if "analysis" in item])
        dir_length.append(length)
        lemmas = len(set(mystem.lemmatize(direction)))
        dir_lemmas.append(lemmas)
    return mean(dir_length), mean(dir_lemmas)

## Putting it all together

### Saving the data
The following is being done in order to save the extraced directions just in case (and not to extract them from the play every time we need them).

In [7]:
# some basic stuff we need in order to save everything properly
directions_path = ".." + os.sep + "data"
if not os.path.exists(directions_path):
    os.mkdirs(directions_path)

all_directions_file = directions_path + os.sep + "all_directions.txt"
if not os.path.exists(all_directions_file):
    f = open(all_directions_file, "w", encoding="utf-8")
    f.close()

In [8]:
def save_directions(play_name, directions, option):
    if option == "sep":
        directions_file = directions_path + os.sep + play_name.replace("xml", "txt")
        with open(directions_file, "w", encoding="utf-8") as directions_save:
            directions_save.write("\n".join(directions))
    elif option == "all":
        with open(all_directions_file, "a", encoding="utf-8") as all_directions_save:
            all_directions_save.write("\n".join(directions) + "\n")

### Working with the corpus

Now, we traverse all the plays in the ```corpus_folder``` and do the following: 
* extract all the directions and save them: 
    * in a ```directions_list``` variable,
    * in a separate file,
    * in a file that contains all the directions
* extract information about a play and save it into ```play_list``` (to convert it into a Pandas dataframe later)

In [9]:
# this is required because I'm using a Mac, which sometimes creates system folders like .DS_Store;
# in any other cases -- never mind
play_files = [item for item in os.listdir(corpus_path) if item.endswith(".xml")]

In [10]:
directions_list = []
play_list = []

dropped_count = 1
for play in play_files:
    play_path = corpus_path + os.sep + play
    
    st_dirs = get_directions(play_path)
    author, title, year_written, year_published, acts = get_play_data(play_path)
    lemmas, words = word_count(play_path, mystem)
    words_dir, lemmas_dir = average_direction(st_dirs, mystem)
    
    play_info = {"File": play, 
            "Author": author, 
            "Title": title, 
            "Written": year_written, 
            "Published": year_published, 
            "Amount of directions": len(st_dirs), 
            "Amount of acts": acts if acts!=0 else 1,
            "Lemmas": lemmas,
            "Words": words,
            "Words per direction": words_dir,
            "Lemmas per direction": lemmas_dir}
    
    if (play_info["Written"] != -1) and (play_info["Published"] != -1):
        # directions
        directions_list.append(st_dirs)
        save_directions(play, st_dirs, option="sep")
        save_directions(play, st_dirs, option="all")
        
        # add information about a play
        play_list.append(play_info)
    else:
        print("Dropped a play: {}, total dropped: {}".format(play_path, dropped_count))
        dropped_count += 1

Dropped a play: ../data_april/sukhovo-kobylin-smert-tarelkina.xml, total dropped: 1
Dropped a play: ../data_april/shakhovskoy-ne-lubo-ne-slushai.xml, total dropped: 2
Dropped a play: ../data_april/shakhovskoy-pustodomy.xml, total dropped: 3
Dropped a play: ../data_april/lomonosov-tamira-i-selim.xml, total dropped: 4
Dropped a play: ../data_april/naydyonov-deti-vanjushina.xml, total dropped: 5
Dropped a play: ../data_april/pisemskij-gorkaja-sudbina.xml, total dropped: 6
Dropped a play: ../data_april/plavilshchikov-sgovor-kutejkina.xml, total dropped: 7
Dropped a play: ../data_april/shakhovskoy-svoya-semya.xml, total dropped: 8
Dropped a play: ../data_april/turgenev-razgovor-na-bolshoj-doroge.xml, total dropped: 9
Dropped a play: ../data_april/prutkov-spor-drevnih-grecheskih-filosofov-ob-izjaschnom.xml, total dropped: 10
Dropped a play: ../data_april/shakhovskoy-urok-koketkam.xml, total dropped: 11
Dropped a play: ../data_april/ozerov-dmitrij-donskoj.xml, total dropped: 12
Dropped a play

#### Corrections
Note that the first ```if``` statement in the ```for``` loop removes all the plays where __it is not clear when the play was written and/or published__.

Furthermore, if there were no ```<div type="act">```, it means that __there is only one act in the play (not zero)__. I corrected it while creating ```play_info```.

### Data frame

Now, we create a Pandas data frame with all the information about the plays. We'll use paths to the plays to refer to rows.

In [11]:
play_df = pd.DataFrame(play_list)
play_df.head()

,Amount of acts,Amount of directions,Author,File,Lemmas,Lemmas per direction,Published,Title,Words,Words per direction,Written
0,1,38,"Пушкин, Александр Сергеевич",pushkin-stseny-iz-rytsarskih-vremen.xml,996,4.973684,1837,Сцены из рыцарских времен,3399,3.131579,1835
1,1,687,"Тургенев, Иван Сергеевич",turgenev-holostjak.xml,2785,6.391557,1849,Холостяк,21501,4.506550,1849
2,2,254,"Гоголь, Николай Васильевич",gogol-zhenitba.xml,2187,5.952756,1842,Женитьба,13094,3.925197,1835
3,1,80,"Блок, Александр Александрович",blok-neznakomka.xml,1342,18.025000,1907,Незнакомка,4222,16.462500,1906
4,1,35,"Гоголь, Николай Васильевич",gogol-lakeiskaja.xml,611,8.428571,1842,Лакейская,1648,6.657143,1840


## Simple numbers
We will calculate:

* word and lemma count,
* amounts of the following parts-of-speech:
    - nouns,
    - adjectives,
    - verbs,
    - adverbs,
    - interjections
* amount of directions, per act

### Word, lemma count
Word and lemma count were calculated previously (see "Extraction" chapter). In here, I also calculate amount of words and lemmas per act.

In [12]:
# directions per act
play_df["Directions per act"] = play_df["Amount of directions"]/play_df["Amount of acts"]
# words per act
play_df["Words per act"] = play_df["Words"]/play_df["Amount of acts"]
# lemmas per act
play_df["Lemmas per act"] = play_df["Lemmas"]/play_df["Amount of acts"]
play_df.head()

,Amount of acts,Amount of directions,Author,File,Lemmas,Lemmas per direction,Published,Title,Words,Words per direction,Written,Directions per act,Words per act,Lemmas per act
0,1,38,"Пушкин, Александр Сергеевич",pushkin-stseny-iz-rytsarskih-vremen.xml,996,4.973684,1837,Сцены из рыцарских времен,3399,3.131579,1835,38.0,3399.0,996.0
1,1,687,"Тургенев, Иван Сергеевич",turgenev-holostjak.xml,2785,6.391557,1849,Холостяк,21501,4.506550,1849,687.0,21501.0,2785.0
2,2,254,"Гоголь, Николай Васильевич",gogol-zhenitba.xml,2187,5.952756,1842,Женитьба,13094,3.925197,1835,127.0,6547.0,1093.5
3,1,80,"Блок, Александр Александрович",blok-neznakomka.xml,1342,18.025000,1907,Незнакомка,4222,16.462500,1906,80.0,4222.0,1342.0
4,1,35,"Гоголь, Николай Васильевич",gogol-lakeiskaja.xml,611,8.428571,1842,Лакейская,1648,6.657143,1840,35.0,1648.0,611.0


### Saving the dataframe

In [13]:
play_df.to_csv(".." + os.sep + "data" + os.sep + "general_information.csv", sep=";", encoding="utf-8")